 # Надо:
* Считать данные из текстового файла
* Переделать в numpy array(2 размерности)
* **ОБЯЗАТЕЛЬНО** нагенерить данных для обучения
* Разбить на выборки
* Обучить модель
* Перевести предсказания в пары

In [9]:
import numpy as np
import pandas as ps

In [27]:
dtt = open('example.txt', 'r')
dt = dtt.readlines()
dat = [] #Будущие данные о пользователях
for usr in dt:
    dat.append(usr.split(',')) #ВАЖНО! разделитель данных -- ЗАПЯТАЯ

''' Данные о пользователе: [id, nickname, room_number, direction, (sports...)]
                           [0,  kllllevo, 13,           1,         0,1,0,0]
'''
#dat
dtt.close()

In [28]:
data = {}
for usr in dat:
    data[usr[0]] = list(usr[1:])

''' Данные о пользователях: {'id':[Данные о пользователе, кроме id]} 
'''
data

{'1': ['dima', '1', '3', '1', '0', '1\n'],
 '10': ['isuss', '2', '3', '1', '1', '1'],
 '2': ['pasha', '1', '4', '0', '1', '1\n'],
 '3': ['dunkan', '4', '3', '0', '0', '0\n'],
 '4': ['fubar', '2', '2', '0', '0', '0\n'],
 '5': ['slava', '1', '4', '1', '1', '1\n'],
 '6': ['vania', '3', '1', '0', '1', '0\n'],
 '7': ['programmer', '1', '2', '0', '0', '1\n'],
 '8': ['hanamury', '3', '1', '1', '0', '0\n'],
 '9': ['alah', '5', '4', '0', '0', '0\n']}

In [29]:
drata = []
for i in list(data.keys()):
    a = list(data.keys())
    a.remove(i)
    drata.append([val for val in a])

''' Данные вида [another users id] -- this for user with id=0
                [another users id] -- for id=1
                [another users id] -- for id=2
'''
#drata

' Данные вида [another users id] -- this for user with id=0\n                [another users id] -- for id=1\n                [another users id] -- for id=2\n'

In [30]:
final_data = []
nmb = 1 # id юзера(см. ниже)
for lst in drata:
    top_ids= [] #здесь хранятся id
    with_ids = [] # тут -- число совпадений
    top = []
    for u in lst:
        u_data = np.array(data[u][1:])# Вся дата того, кого проверяем, кроме nickname
        usr_data = np.array(data[str(nmb)][1:])# Вся дата того, ДЛЯ кого проверяем, кроме nickname
        with_ids.append(len(np.where(u_data == usr_data)[0]))
        top_ids.append(u)
    args_by_sort = np.argsort(np.array(with_ids))
    #print('top_ids',top_ids)
    #print('with_ids',with_ids)
    #print('args_by_sort',args_by_sort)
    for arg in args_by_sort:
        top.append(top_ids[arg])
    final_data.append(top)
    nmb += 1
'''Вот это б*ть то, ради чего мы все тут собрались. Данная клетка воплощает идею о том,
Чтобы составить все возможные пары киллер-жертва, и отсортировать потенциальных жертв по
кол-ву совпадений по признакам. Выхлоп -- внизу
'''

'Вот это б*ть то, ради чего мы все тут собрались. Данная клетка воплощает идею о том,\nЧтобы составить все возможные пары киллер-жертва, и отсортировать потенциальных жертв по\nкол-ву совпадений по признакам. Выхлоп -- внизу\n'

In [31]:
final_data

[['6', '4', '9', '2', '3', '8', '10', '5', '7'],
 ['8', '3', '4', '10', '1', '6', '9', '7', '5'],
 ['5', '2', '10', '1', '6', '7', '8', '4', '9'],
 ['5', '1', '2', '10', '6', '8', '3', '7', '9'],
 ['3', '4', '6', '8', '9', '7', '10', '1', '2'],
 ['1', '5', '7', '10', '2', '3', '4', '9', '8'],
 ['10', '6', '8', '3', '5', '9', '1', '2', '4'],
 ['2', '5', '7', '10', '1', '3', '4', '9', '6'],
 ['10', '1', '5', '2', '6', '7', '8', '3', '4'],
 ['7', '9', '2', '3', '4', '6', '8', '1', '5']]

In [32]:
weighted_data = []
nmb = 1
for lst in final_data:#Не забыть nmb += 1
    weighted = []
    cloned_final = list(final_data)
    cloned_final.remove(lst)
    for u in lst:   
        weight = 0
        for sub_lst in cloned_final:
            for sub_u in sub_lst:
                if sub_lst.index(sub_u) <= lst.index(u) and u == sub_u:
                    weight += lst.index(u) - sub_lst.index(sub_u) + 1
        weighted.append(weight)
    weighted_data.append(weighted)
''' Данная чудо-ячейка возвращает нам матрицу с "весами" id ,где каждому элементу этого массива
сопоставляется id из final_data
'''
weighted_data

[[0, 1, 2, 12, 13, 14, 36, 40, 39],
 [0, 2, 4, 13, 16, 22, 16, 30, 49],
 [1, 2, 6, 10, 14, 16, 22, 28, 32],
 [1, 3, 6, 13, 14, 14, 28, 30, 32],
 [0, 1, 5, 6, 7, 16, 36, 38, 50],
 [0, 5, 4, 13, 19, 20, 21, 23, 39],
 [1, 2, 3, 8, 21, 11, 29, 41, 37],
 [0, 5, 4, 13, 16, 20, 21, 23, 47],
 [1, 3, 10, 12, 14, 16, 22, 37, 37],
 [0, 0, 6, 8, 11, 22, 22, 38, 49]]

In [33]:
%%time
pair = []
for n in range(0, len(final_data)):
    ind = np.argmin(weighted_data[n])
    cdel = int(final_data[n][ind]) - 1
    for j in range(len(final_data[cdel])):
        if final_data[cdel][j] == str(n + 1):
            del weighted_data[cdel][j]
            del final_data[cdel][j]
            break
    pair.append(cdel + 1)
    for i in range(len(final_data)):
        if i != n and i != cdel:
            for j in range(0, len(final_data[i])):
                if final_data[i][j] == str(cdel + 1):
                    del final_data[i][j]
                    del weighted_data[i][j]
                    break
print(pair)

[6, 8, 5, 1, 4, 7, 10, 3, 2, 9]
Wall time: 1 ms
